# Digits Recognizer Kaggle competition
### Author: Guillermo Campollo Ramirez
#### 4/19/2020

## Importing libraries

In [36]:
import pandas as pd
from keras import models
from keras import layers
import numpy as np
from keras.datasets import mnist
from keras.utils import to_categorical

In [53]:
test_set = pd.read_csv('/Users/campo/Documents/Kaggle/data/raw/digits_test.csv')
test_set = np.array(test_set).reshape(28000,28,28,1)
test_set = test_set.astype('float32') / 255

## Creating model object and adding layers

In [44]:
from keras.layers import Input,Dense, Conv2D, Dropout, MaxPool2D, Flatten
# Adding layers
input_layer = Input(shape = (28,28,1),dtype = 'float32')
conv1 = Conv2D(32,(3,3),activation = 'relu')(input_layer)
maxpool1 = MaxPool2D((2,2))(conv1)
conv2 = Conv2D(64,(3,3),activation = 'relu',dtype = 'float32')(maxpool1)
maxpool2 = MaxPool2D((2,2))(conv2)
conv3 = Conv2D(64,(3,3),activation = 'relu',dtype = 'float32')(maxpool2)
flatten = Flatten()(conv3)
dense1 = Dense(64,activation = 'relu')(flatten)
output_layer = Dense(10,activation = 'softmax')(dense1)


model = models.Model(inputs = input_layer, outputs = output_layer)
# Compile the model
model.compile(optimizer = 'rmsprop',
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 576)               0   

## Splitting our dataset into training and testing

In [45]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Fitting our model

In [46]:
model.fit(train_images,train_labels,epochs=5,batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 44s 729us/step - loss: 0.1724 - accuracy: 0.9462
Epoch 2/5
60000/60000 [==============================] - 44s 737us/step - loss: 0.0458 - accuracy: 0.9859
Epoch 3/5
60000/60000 [==============================] - 44s 740us/step - loss: 0.0329 - accuracy: 0.9898
Epoch 4/5
60000/60000 [==============================] - 45s 753us/step - loss: 0.0249 - accuracy: 0.9922
Epoch 5/5
60000/60000 [==============================] - 46s 762us/step - loss: 0.0194 - accuracy: 0.9942


## Predicting our model on test set

In [47]:
test_loss, test_accuracy = model.evaluate(test_images,test_labels)

10000/10000 [==============================] - 2s 240us/step


In [48]:
test_accuracy

0.9904999732971191

## Our model performed well, now we pass the results into single scalars

In [55]:
y_pred=model.predict(test_set) #Variable containing arrays of predictions

In [56]:
y_kaggle=[0]*len(y_pred) # Variable with scalars versions of predictions
for i in range(len(y_pred)):
    y_kaggle[i]=int(np.argmax(y_pred[i]))
        


In [57]:
results=pd.DataFrame({"ImageId":range(1,len(y_kaggle)+1),"Label":y_kaggle})

In [92]:
## Passing our dataframe to CSV format for submission

In [59]:
results.to_csv('../predictions/Digits.csv')